In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

import warnings

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
df = pd.read_csv("../data/processed/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";")

## 1. Análisis de sentimiento

En este cuadernillo se quiere responder las siguientes preguntas:

1. ¿Cuál es el sentimiento predominante en cada una de los detalles de atención?
2. ¿Existe alguna tendencia de estos sentimientos según la especialidad donde se atiende el paciente?
3. ¿Existe alguna tendencia de estos sentimientos según el diagnóstico que se le asignó al paciente?

### 1.1 Análisis Exploratorio de los datos

In [ ]:
vectorizador = CountVectorizer()
palabras_a_filtrar = stopwords.words("spanish")

atenciones = df["detalle_atencion"].dropna().str.lower()
conteo_de_palabras = vectorizador.fit_transform(atenciones)

In [ ]:
frecuencias = conteo_de_palabras.sum(axis=0)
frecuencias = frecuencias.tolist()[0]

In [ ]:
word_freq_df = pd.DataFrame(
    {"Word": vectorizador.get_feature_names_out(), "Frequency": frecuencias}
).sort_values("Frequency", ascending=False)

In [ ]:
word_freq_df.head(30).plot(y="Frequency", x="Word", kind="bar")

El análisis de las 30 palabras mas frecuentes indican lo siguiente:

- Gran parte de las palabras más frecuentes corresponden a conjunciones (de, en, con, no, se, sin, etc).
Por lo tanto, es necesario filtrar estas palabras.

- Existen palabras de alta relevancia, tales como: control, tac, pulmonar, etc...

Por lo tanto, es necesario limpiar los textos

In [ ]:
df["detalle_atencion_procesada"] = build_features.preprocess_text(df["detalle_atencion"], "spanish")

In [ ]:
vectorizador_procesado = CountVectorizer()
matriz_conteo = vectorizador_procesado.fit_transform(df["detalle_atencion_procesada"])
conteo_palabras_procesadas = matriz_conteo.sum(axis=0).tolist()[0]

In [ ]:
df_conteo_palabras_procesadas = pd.DataFrame(
    {
        "Word": vectorizador_procesado.get_feature_names_out(),
        "Frequency": conteo_palabras_procesadas,
    }
)
df_conteo_palabras_procesadas = df_conteo_palabras_procesadas.sort_values(
    "Frequency", ascending=False
)

In [ ]:
sns.set_color_codes("pastel")
sns.barplot(data=df_conteo_palabras_procesadas.head(30), x="Frequency", y="Word", color="b")

Una vez realizado el preprocesamiento de los textos analizados se observa que la palabra más frecuente
es control. Otras palabras interesante que son altamente frecuente son: "tac", "pulmonar", "mg", "disnea", entre otras.